In [1]:
import datetime
import os
import numpy as np
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go
init_notebook_mode(connected=True)

from obs_data import *
from plotlyFunctions import *

print(os.getcwd())

2016
D:\Documents\these_pablo\Models\BEACH2016\Analysis


### Path and working computer

In [2]:
PC = True
analysis = False
short_sim = True
test = 'test_Epsilon'
version = "mFix_v1"
folders = 3
timestamp = 'julian'

In [3]:
# Path to simulated folder
if analysis:
    dot = '.'
else:
    dot = '..'

# Working computer
if PC:
    back = "\\"  
    sim_path = dot + back 
else:
    back = "/"
    sim_path = dot + back 
    
sim_path += test + back + version + back  
    
print(sim_path)

..\test_Epsilon\mFix_v1\


In [4]:
# Function to add column ID, with first letter capitalized.
def newlabel(row, plot):
    return plot.capitalize() + '-' + str(int(row['Jdays']))

# Funcrion add simulation ID
def simulation(folder):
    return 'Sim' + '-' + str(folder)

In [5]:
# TODO: Change t -> v (also in the observed data: 0.2_XYcalibrationSoils.Rmd)
detail_plots = ['n1', 'n2', 'n3', 'n4', 'n5', 'n7', 'n8',
                'v4', 'v5', 'v7', 'v8', 'v9', 'v10',
                's11', 's12', 's13']

sample_comp = ['nor', 'val', 'sou']

## Concentrations

In [6]:
variable = 'CONC'
name_obs = 'conc_det_cal'
matches = []

for folder in range(1, folders + 1):
    for sample in range(len(detail_plots)):
        plot = detail_plots[sample]
        filename = "resM_" + plot + variable + ".tss"  # Simulated
        det_sim = pd.read_table(sim_path + str(folder) + back + filename, 
                                    skiprows=4, delim_whitespace=True,
                                    names=['Jdays', 'SIM'],
                                    header=None)
        # Add ID
        det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, plot),axis=1) 
        det_sim['IDsim'] = det_sim.apply (lambda row: simulation(folder), axis=1)
        n = getSoilDataCal(name_obs, tss=True)  # Observed

        match = pd.merge(n, det_sim, how='inner', on='IDcal')
        matches.append(match)
    
# Concatenate detailed
conc_det_dat = pd.concat(matches)

In [7]:
# Prepare composites and concatenate
variable = 'CONC'
name_obs = 'conc_comp_cal'
matches = []
for folder in range(1, folders + 1):
    for sample in range(len(sample_comp)):
        plot = sample_comp[sample]
        filename = "resM_" + plot + variable + ".tss"
        det_sim = pd.read_table(sim_path + str(folder) + back + filename, 
                                    skiprows=4, delim_whitespace=True,
                                    names=['Jdays', 'SIM'],
                                    header=None)
        # Add ID (remove after running model)
        #if sample == 1:
        #    label = "T"
        #else:
        #    label = plot[0]
        label = plot[0]
        det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, label),axis=1) 
        det_sim['IDsim'] = det_sim.apply (lambda row: simulation(folder), axis=1)
        n = getSoilDataCal(name_obs, tss=True)
        match = pd.merge(n, det_sim, how='inner', on='IDcal')
        matches.append(match)
        
# Concatenate composites
conc_comp_dat = pd.concat(matches)

conc_dat = pd.concat([conc_det_dat, conc_comp_dat])

In [8]:
conc_dat.head()

,IDcal,IDsim,Jdays_x,Jdays_y,SIM,Transect,ug.g,ug.g.SD
0,N1-181,Sim-1,181,181,0.05728,N,0.071209,NaN
1,N1-286,Sim-1,286,286,4.24090,N,4.289449,NaN
0,N2-181,Sim-1,181,181,6.76770,N,4.710027,NaN
1,N2-225,Sim-1,225,225,2.36769,N,4.781827,NaN
2,N2-272,Sim-1,272,272,1.21290,N,0.776233,NaN


## Delta

In [9]:
variable = 'd13C'
name_obs = 'delta_det_cal'
matches = []
for folder in range(1, folders + 1):
    for sample in range(len(detail_plots)):
        plot = detail_plots[sample]
        filename = "resM_" + plot + variable + ".tss"
        det_sim = pd.read_table(sim_path + str(folder) + back + filename, 
                                    skiprows=4, delim_whitespace=True,
                                    names=['Jdays', 'SIM'],
                                    header=None)
        # Add ID
        det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, plot),axis=1) 
        det_sim['IDsim'] = det_sim.apply (lambda row: simulation(folder), axis=1)
        n = getSoilDataCal(name_obs, tss=True)
        match = pd.merge(n, det_sim, how='inner', on='IDcal')
        matches.append(match)
    
delta_det_dat = pd.concat(matches)

name_obs = 'delta_comp_cal'
matches = []
for folder in range(1, folders + 1):
    for sample in range(len(sample_comp)):
        plot = sample_comp[sample]
        filename = "resM_" + plot + variable + ".tss"
        det_sim = pd.read_table(sim_path + str(folder) + back + filename, 
                                    skiprows=4, delim_whitespace=True,
                                    names=['Jdays', 'SIM'],
                                    header=None)
        det_sim['IDcal'] = det_sim.apply (lambda row: newlabel (row, plot[0]),axis=1) 
        det_sim['IDsim'] = det_sim.apply (lambda row: simulation(folder), axis=1)
        n = getSoilDataCal(name_obs, tss=True)
        match = pd.merge(n, det_sim, how='inner', on='IDcal')
        matches.append(match)
    
delta_comp_dat = pd.concat(matches)

delta_dat = pd.concat([delta_det_dat, delta_comp_dat])

In [10]:
delta_dat.head()

,Jdays_x,Transect,IDcal,d13C,d13C.SD,Jdays_y,SIM,IDsim
0,181,N,N2-181,-30.663750,0.113573,181,-32.1132,Sim-1
0,197,N,N7-197,-30.846000,0.055830,197,-32.2122,Sim-1
1,244,N,N7-244,-28.795500,0.255266,244,-31.7498,Sim-1
0,244,N,N8-244,-28.855000,0.106066,244,-31.7500,Sim-1
0,197,V,V4-197,-31.971667,0.604105,197,-32.2116,Sim-1


## Plotting

### Concentrations

In [11]:
conc_dat.head()

,IDcal,IDsim,Jdays_x,Jdays_y,SIM,Transect,ug.g,ug.g.SD
0,N1-181,Sim-1,181,181,0.05728,N,0.071209,NaN
1,N1-286,Sim-1,286,286,4.24090,N,4.289449,NaN
0,N2-181,Sim-1,181,181,6.76770,N,4.710027,NaN
1,N2-225,Sim-1,225,225,2.36769,N,4.781827,NaN
2,N2-272,Sim-1,272,272,1.21290,N,0.776233,NaN


In [12]:
test_var = 'ug.g'
sd = 'ug.g.SD'

data = []
for folder in range(1, folders + 1):
    df = conc_dat.loc[conc_dat['IDsim'] == ('Sim-' + str(folder))]
    #df = df.loc[df['Transect'] == 'V']
    trace = get_scatter(df, test_var, sd, name=('Sim-' + str(folder)))
    data.append(trace)
    
data.append(get_xy_line(conc_dat, test_var))

fig = go.Figure(data=data, layout=get_layout(conc_dat, test_var, folders, title='Conc. Soils (Sim vs. Obs)'))
iplot(fig)

### Isotopes

In [13]:
delta_dat.tail()

,Jdays_x,Transect,IDcal,d13C,d13C.SD,Jdays_y,SIM,IDsim
6,230,S,S-230,-29.466667,0.642910,230,-31.1015,Sim-3
7,244,S,S-244,-28.633333,0.288675,244,-31.0307,Sim-3
8,251,S,S-251,-28.933333,0.152753,251,-31.0098,Sim-3
9,258,S,S-258,-27.600000,0.435890,258,-30.9949,Sim-3
10,265,S,S-265,-28.166667,0.378594,265,-30.9843,Sim-3


In [14]:
test_var = 'd13C'
sd = 'd13C.SD'

data = []
for folder in range(1, folders + 1):
    df = delta_dat.loc[delta_dat['IDsim'] == ('Sim-' + str(folder))]
    #df = df.loc[df['Transect'] == 'N']
    trace = get_scatter(df, test_var, sd, name=('Sim-' + str(folder)))
    data.append(trace)
    
data.append(get_xy_line(delta_dat, test_var, delta=True))

fig = go.Figure(data=data, layout=get_layout(delta_dat, test_var, folders, title='Delta Soils (Sim vs. Obs)', delta=True))
iplot(fig)

In [15]:
data = []
df = delta_dat.loc[delta_dat['Transect'] == 'N']
for folder in range(1, folders + 1):
    df1 = df.loc[df['IDsim'] == ('Sim-' + str(folder))]
    trace = get_scatter(df1, test_var, sd, name=('Sim-' + str(folder)))
    data.append(trace)
    
data.append(get_xy_line(df, test_var, delta=True))

fig = go.Figure(data=data, layout=get_layout(df, test_var, folders, title='North Delta Soils (Sim vs. Obs)', delta=True))
iplot(fig)

In [16]:
data = []
df = delta_dat.loc[delta_dat['Transect'] == 'V']
for folder in range(1, folders + 1):
    df1 = df.loc[df['IDsim'] == ('Sim-' + str(folder))]
    trace = get_scatter(df1, test_var, sd, name=('Sim-' + str(folder)))
    data.append(trace)
    
data.append(get_xy_line(df, test_var, delta=True))

fig = go.Figure(data=data, layout=get_layout(df, test_var, folders, title='Valley Delta Soils (Sim vs. Obs)', delta=True))
iplot(fig)

In [17]:
data = []
df = delta_dat.loc[delta_dat['Transect'] == 'S']
for folder in range(1, folders + 1):
    df1 = df.loc[df['IDsim'] == ('Sim-' + str(folder))]
    trace = get_scatter(df1, test_var, sd, name=('Sim-' + str(folder)))
    data.append(trace)
    
data.append(get_xy_line(df, test_var, delta=True))

fig = go.Figure(data=data, layout=get_layout(df, test_var, folders, title='South Delta Soils (Sim vs. Obs)', delta=True))
iplot(fig)

In [18]:
delta_dat.head()

,Jdays_x,Transect,IDcal,d13C,d13C.SD,Jdays_y,SIM,IDsim
0,181,N,N2-181,-30.663750,0.113573,181,-32.1132,Sim-1
0,197,N,N7-197,-30.846000,0.055830,197,-32.2122,Sim-1
1,244,N,N7-244,-28.795500,0.255266,244,-31.7498,Sim-1
0,244,N,N8-244,-28.855000,0.106066,244,-31.7500,Sim-1
0,197,V,V4-197,-31.971667,0.604105,197,-32.2116,Sim-1
